### Import and webdriver setup

In [ ]:
#C:/Users/grema/Jupyter python stuff/PlayStoreScraper/chromedriver.exe
#C:/Users/jackie.yu/Documents/WeCloudData/Python/PlayStoreProject/PlayStoreScraper/chromedriver.exe

In [ ]:
import re
import random
from time import sleep
from datetime import datetime

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

driver = webdriver.Chrome('C:/Users/jackie.yu/Documents/WeCloudData/Python/PlayStoreProject/PlayStoreScraper/chromedriver.exe')
driver.get("https://play.google.com/store/apps/details?id=cc.forestapp&showAllReviews=true")

### Parsing through the html with Selenium

In [ ]:
# Expands the "Read more" button for reviews and scrolls
for _ in range(20):   
    button_list = driver.find_elements_by_css_selector("button[jsname='gxjVle']")
    for button in button_list:
        driver.execute_script("arguments[0].click();", button)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    sleep(random.randint(0,10))
    try:
        driver.find_element_by_class_name("U26fgb.O0WRkf.oG5Srb.C0oVfc.n9lfJ.M9Bg4d").click()
    
        print("i clicked")
    except:
        print("me pass")
        pass

In [ ]:
# Grabs the non-expandable reviews and puts them into a dictionary with an index
# Expandable reviews will show up as an empty space in the dictionary due to "display = none" on the css
list_of_reviews = driver.find_elements_by_css_selector("span[jsname='bN97Pc']")
review_dict = {}
for i, review in enumerate(list_of_reviews):
    review_dict[i] = review.text
review_dict

In [ ]:

# Grabs the expandable reviews and puts them into a dictionary with an index
# Non-Expandable reviews will show up as an empty space in the dictionary due to "display = none" on the css
list_of_reviews2 = driver.find_elements_by_css_selector("span[jsname='fbQN7e']")
expanded_review_dict = {}
for i, review in enumerate(list_of_reviews2):
    expanded_review_dict[i] = review.text
expanded_review_dict

In [ ]:
# Due to the nature of the dictionaries, we can replace the empty space with values from the opposite dictionary
# to make the full list of the review
# index is required to match with other data like stars and date
for i in review_dict:
    if review_dict[i] == '':
        review_dict[i] = expanded_review_dict[i]
review_dict

In [ ]:
#filters out developer responses
list_of_users = driver.find_elements_by_class_name("bAhLNe.kx8XBd")

list_of_dates = [user.find_elements_by_class_name('p2TkOb')[0].text for user in list_of_users]

date_dict = {}

for i, date in enumerate(list_of_dates):
    date_dict[i] = datetime.strptime(date, '%B %d, %Y')
    #date_dict[i].strftime('%Y-%m-%d') to convert back to string
date_dict


In [ ]:
list_of_likes = driver.find_elements_by_class_name('jUL89d.y92BAb')
like_dict = {}

for i, like in enumerate(list_of_likes):
    if like.text == '':
        like_dict[i] = 0
    else:
        like_dict[i] = int(like.text)
like_dict

In [ ]:
# Parent class containing the star review
star_parent_class = driver.find_elements_by_class_name("pf5lIe")
# Child class containing the string star rating e.g. "Rated 5 stars out of five stars"
list_of_stars = [parent.find_element_by_css_selector("div").get_attribute("aria-label") for parent in star_parent_class]
star_dict = {}

for i, star in enumerate(list_of_stars):
    # Retrieving reviews that do not have a float value 
    # as these come from the recommended apps on the bottom
    if not re.search('(\d\.\d)', star):        
        star_dict[i-1] = int(re.search('[0-5]', star).group())

star_dict

In [ ]:
forest_list = [date_dict, star_dict, review_dict, like_dict]
forest_list